In [ ]:
from google import genai
from google.genai import types

import pandas as pd

import time

import os

In [2]:
qaSet_interpret = pd.read_csv('./Dataset/qa_set_interpret.csv', encoding='utf-8-sig')
qaSet_engineer = pd.read_csv('./Dataset/qa_set.csv', encoding='utf-8-sig')
# mefi_list = pd.read_csv('MeFi_result.csv', encoding='utf-8-sig')

In [ ]:
Interpreter_inst = "You are designed to assist users in interpreting legal provisions related to the Korean Architecture Code. Your primary role is to provide interpretations of the laws as they apply to inquiries in Korean. The answer must consist of 3 parts: ‘질의요지’, ‘종합결론’, ‘판단근거’. The first part, ‘질의요지’, describes the core content of the inquiry. The second part, ‘종합결론’, describes the answer or conclusion of the question. The last part, ‘판단근거’, contains the legal basis of the answer described in ‘Comprehensive Conclusion’."

# You may change the instruction to suit your needs. For example, you can use below instruction for BEQA: 
Extractive_inst = "Return only the answer number first like Answer: [number], and then explain the reason."

## 1. Gemini 2.5 Flash

### 1-1. Interpret

In [ ]:
# You should have your own api key
client = genai.Client(api_key="")
max_len = len(qaSet_interpret)
print(max_len)
writeFile = './result/AutoCon_Interpret_Gemini2.5f.csv'

In [ ]:
for q_num in range(0, max_len):
    print(f"-----Question {q_num+1}(Index: {q_num}) Start-----")
    start = time.time()
    
    response = client.models.generate_content(
        model='gemini-2.5-flash-preview-04-17',
        contents=[Interpreter_inst, qaSet_interpret['qeustion'][q_num]]
    )
    print(response.text)

    end = time.time()
    meta = str(response.usage_metadata) + "\nExecution Time: "+str(end - start)

    responseRecord = pd.read_csv(writeFile, dtype='string', encoding='utf-8-sig')
    responseRecord.loc[q_num, 'metadata'] = meta
    responseRecord.loc[q_num, 'response'] = response.text
    responseRecord.to_csv(writeFile, encoding='utf-8-sig', index=False)

    print(f"Completed Responsing. Took {end - start:.5f} sec")
    print(f"-----Question {q_num+1}(Index: {q_num}) End-----")

### 1-2. Extractive Comprehension

In [ ]:
writeFile = './result/AutoCon_PlainTest_Gemini2.5f.csv'

In [ ]:
for q_num in range(0, len(qaSet_engineer)):
    print(f"-----Question {q_num+1}(Index: {q_num}) Start-----")
    start = time.time()
    
    response = client.models.generate_content(
        model='gemini-2.5-flash-preview-04-17',
        contents=[Extractive_inst, qaSet_engineer['qeustion'][q_num]]
    )
    print(response.text)

    end = time.time()
    meta = str(response.usage_metadata) + "\nExecution Time: "+str(end - start)

    responseRecord = pd.read_csv(writeFile, dtype='string', encoding='utf-8-sig')
    responseRecord.loc[q_num, 'metadata'] = meta
    responseRecord.loc[q_num, 'response'] = response.text
    responseRecord.to_csv(writeFile, encoding='utf-8-sig', index=False)

    print(f"Completed Responsing. Took {end - start:.5f} sec")
    print(f"-----Question {q_num+1}(Index: {q_num}) End-----")

## 2. GPT

### 2-1. Interpret

In [ ]:
import openai

# You should have your own api key
os.environ['OPENAI_API_KEY'] = ""
client = openai.OpenAI()

In [ ]:
max_len = len(qaSet_interpret)
writeFile = './result/AutoCon_Interpret_GPT4.1.csv'

In [38]:
for q_num in range(0, max_len):
    print(f"-----Question {q_num+1}(Index: {q_num}) Start-----")
    start = time.time()

    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {
                "role": "system",
                "content": Interpreter_inst
            },
            {
                "role": "user",
                "content": qaSet_interpret['qeustion'][q_num]
            }
        ]
    )
    output_text = response.choices[0].message.content
    print(output_text)

    end = time.time()
    meta = str(response.usage) + "\nExecution Time: "+str(end - start)

    responseRecord = pd.read_csv(writeFile, dtype='string', encoding='utf-8-sig')
    responseRecord.loc[q_num, 'metadata'] = meta
    responseRecord.loc[q_num, 'response'] = output_text
    responseRecord.to_csv(writeFile, encoding='utf-8-sig', index=False)

    print(f"Completed Responsing. Took {end - start:.5f} sec")
    print(f"-----Question {q_num+1}(Index: {q_num}) End-----")

### 2-2. Extractive Comprehension

In [ ]:
writeFile = './result/AutoCon_PlainTest_GPT4.1.csv'
max_len = len(qaSet_engineer)
print(max_len)

In [ ]:
for q_num in range(0, max_len):
    print(f"-----Question {q_num+1}(Index: {q_num}) Start-----")
    start = time.time()

    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {
                "role": "system",
                "content": Extractive_inst
            },
            {
                "role": "user",
                "content": qaSet_engineer['qeustion'][q_num]
            }
        ]
    )
    output_text = response.choices[0].message.content
    print(output_text)

    end = time.time()
    meta = str(response.usage) + "\nExecution Time: "+str(end - start)

    responseRecord = pd.read_csv(writeFile, dtype='string', encoding='utf-8-sig')
    responseRecord.loc[q_num, 'metadata'] = meta
    responseRecord.loc[q_num, 'response'] = output_text
    responseRecord.to_csv(writeFile, encoding='utf-8-sig', index=False)

    print(f"Completed Responsing. Took {end - start:.5f} sec")
    print(f"-----Question {q_num+1}(Index: {q_num}) End-----")

## 3. Claude

### 3-1. Interpret

In [ ]:
import anthropic

client = anthropic.Anthropic(
    # You should have your own api key
    api_key="",
)

In [59]:
writeFile = './result/AutoCon_Interpret_Claude3.7_2.csv'
max_len = len(qaSet_interpret)

In [60]:
for q_num in range(0, max_len):
    print(f"-----Question {q_num+1}(Index: {q_num}) Start-----")
    start = time.time()

    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=2048,
        system=Interpreter_inst,
        messages=[
            {"role": "user", "content": qaSet_interpret['qeustion'][q_num]}
        ]
    )

    print(response.content[0].text)

    end = time.time()
    meta = f"Input Tokens: {response.usage.input_tokens}, Output Tokens: {response.usage.output_tokens},\nExecution Time: {end - start} sec"

    responseRecord = pd.read_csv(writeFile, dtype='string', encoding='utf-8-sig')
    responseRecord.loc[q_num, 'metadata'] = meta
    responseRecord.loc[q_num, 'response'] = response.content[0].text
    responseRecord.to_csv(writeFile, encoding='utf-8-sig', index=False)

    print(f"Completed Responsing. Took {end - start:.5f} sec")
    print(f"-----Question {q_num+1}(Index: {q_num}) End-----")

### 3-2. Extractive Comprehension

In [ ]:
writeFile = './result/AutoCon_PlainTest_Claude3.7.csv'
max_len = len(qaSet_engineer)
print(max_len)

In [ ]:
for q_num in range(0, max_len):
    print(f"-----Question {q_num+1}(Index: {q_num}) Start-----")
    start = time.time()

    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=2048,
        system=Extractive_inst,
        messages=[
            {"role": "user", "content": qaSet_engineer['qeustion'][q_num]}
        ]
    )

    print(response.content[0].text)

    end = time.time()
    meta = f"Input Tokens: {response.usage.input_tokens}, Output Tokens: {response.usage.output_tokens},\nExecution Time: {end - start} sec"

    responseRecord = pd.read_csv(writeFile, dtype='string', encoding='utf-8-sig')
    responseRecord.loc[q_num, 'metadata'] = meta
    responseRecord.loc[q_num, 'response'] = response.content[0].text
    responseRecord.to_csv(writeFile, encoding='utf-8-sig', index=False)

    print(f"Completed Responsing. Took {end - start:.5f} sec")
    print(f"-----Question {q_num+1}(Index: {q_num}) End-----")

## 4. Gemini1.5Pro

### 4-1. Interpret

In [ ]:
# You should have your own api key
client = genai.Client(api_key="")
max_len = len(qaSet_interpret)
print(max_len)
writeFile = './result/AutoCon_Interpret_Gemini1.5Pro.csv'

In [ ]:
for q_num in range(0, max_len):
    print(f"-----Question {q_num+1}(Index: {q_num}) Start-----")
    start = time.time()
    
    response = client.models.generate_content(
        model='gemini-1.5-pro-001',
        contents=[Interpreter_inst, qaSet_interpret['qeustion'][q_num]]
    )
    print(response.text)

    end = time.time()
    meta = str(response.usage_metadata) + "\nExecution Time: "+str(end - start)

    responseRecord = pd.read_csv(writeFile, dtype='string', encoding='utf-8-sig')
    responseRecord.loc[q_num, 'metadata'] = meta
    responseRecord.loc[q_num, 'response'] = response.text
    responseRecord.to_csv(writeFile, encoding='utf-8-sig', index=False)

    print(f"Completed Responsing. Took {end - start:.5f} sec")
    print(f"-----Question {q_num+1}(Index: {q_num}) End-----")

### 4-2. Extractive Comprehension

In [ ]:
writeFile = './result/AutoCon_PlainTest_Gemini1.5Pro.csv'

In [ ]:
for q_num in range(0, len(qaSet_engineer)):
    print(f"-----Question {q_num+1}(Index: {q_num}) Start-----")
    start = time.time()
    
    response = client.models.generate_content(
        model='gemini-1.5-pro-002',
        contents=[Extractive_inst, qaSet_engineer['qeustion'][q_num]]
    )
    print(response.text)

    end = time.time()
    meta = str(response.usage_metadata) + "\nExecution Time: "+str(end - start)

    responseRecord = pd.read_csv(writeFile, dtype='string', encoding='utf-8-sig')
    responseRecord.loc[q_num, 'metadata'] = meta
    responseRecord.loc[q_num, 'response'] = response.text
    responseRecord.to_csv(writeFile, encoding='utf-8-sig', index=False)

    print(f"Completed Responsing. Took {end - start:.5f} sec")
    print(f"-----Question {q_num+1}(Index: {q_num}) End-----")